In [1]:
from DDCASPT2 import DDCASPT2
import pickle, os, shutil
from glob import glob
import numpy as np
from joblib import Parallel, delayed
import pandas as pd
from math import sin, cos, pi
from tqdm.notebook import tqdm
# Geom manipulate
from AaronTools.geometry import Geometry
from tqdm import tqdm
from time import perf_counter

Take [Fe$^{IV}$(O)(H$_2$O)$_4$]$^{+2}$ from Phys. Chem. Chem. Phys., 2018,20, 28786-28795

In [2]:
# radius_range_dirs = []
# for i in glob('*.*'):
#     try:
#         radstr = float(i)
#         if radstr>=1.4:
#             radius_range_dirs.append(i)
#     except:
#         continue

# radius_range = sorted(radius_range_dirs)

# train_ind,test_ind=radius_range[0::2],radius_range[1::2]

# print(len(train_ind),len(test_ind))
# with open('train_ind.pickle', 'wb') as handle:
#     pickle.dump(train_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('test_ind.pickle', 'wb') as handle:
#     pickle.dump(test_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('test_ind.pickle', 'rb') as handle:
#     test_ind = pickle.load(handle)

# with open('train_ind.pickle', 'rb') as handle:
#     train_ind = pickle.load(handle)
    
# print(len(train_ind),len(test_ind))    

In [3]:
radius_range=np.linspace(1.4,8,500)



train_ind,test_ind=radius_range[0::2],radius_range[1::2]
# train_test_split(radius_range, test_size=0.3, random_state=0)
print(len(train_ind),len(test_ind))
with open('train_ind.pickle', 'wb') as handle:
    pickle.dump(train_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('test_ind.pickle', 'wb') as handle:
    pickle.dump(test_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('test_ind.pickle', 'rb') as handle:
    test_ind = pickle.load(handle)

with open('train_ind.pickle', 'rb') as handle:
    train_ind = pickle.load(handle)
    
print(len(train_ind),len(test_ind))    

250 250
250 250


In [4]:
radius_range

array([1.4       , 1.41322645, 1.42645291, 1.43967936, 1.45290581,
       1.46613226, 1.47935872, 1.49258517, 1.50581162, 1.51903808,
       1.53226453, 1.54549098, 1.55871743, 1.57194389, 1.58517034,
       1.59839679, 1.61162325, 1.6248497 , 1.63807615, 1.65130261,
       1.66452906, 1.67775551, 1.69098196, 1.70420842, 1.71743487,
       1.73066132, 1.74388778, 1.75711423, 1.77034068, 1.78356713,
       1.79679359, 1.81002004, 1.82324649, 1.83647295, 1.8496994 ,
       1.86292585, 1.8761523 , 1.88937876, 1.90260521, 1.91583166,
       1.92905812, 1.94228457, 1.95551102, 1.96873747, 1.98196393,
       1.99519038, 2.00841683, 2.02164329, 2.03486974, 2.04809619,
       2.06132265, 2.0745491 , 2.08777555, 2.101002  , 2.11422846,
       2.12745491, 2.14068136, 2.15390782, 2.16713427, 2.18036072,
       2.19358717, 2.20681363, 2.22004008, 2.23326653, 2.24649299,
       2.25971944, 2.27294589, 2.28617234, 2.2993988 , 2.31262525,
       2.3258517 , 2.33907816, 2.35230461, 2.36553106, 2.37875

In [5]:
def gen_run(r,path):
    with open(os.path.join(path,f'run.sh'),'w') as g:
        g.write(f"""#!/bin/bash
#This file is a submission script to request the ISAAC resources from Slurm
#SBATCH --account=ACF-UTK0022             # The project account to be charged
#SBATCH --job-name=ironoxo_{r}		       #The name of the job
#SBATCH --nodes=1                     # Number of nodes
#SBATCH --ntasks-per-node=16          # cpus per node
#SBATCH --partition=condo-kvogiatz            # If not specified then default is "campus"
#SBATCH --qos=condo-kvogiatz
#SBATCH --time=1-00:00:00             # Wall time (days-hh:mm:ss)
#SBATCH --error=job.e%J	       # The file where run time errors will be dumped
#SBATCH --output=job.o%J	       # The file where the output of the terminal will be dumped



# PATCH!
# MOVE TO WORKING DIR
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR
pwd
# DEFINE WorkDir
mkdir $SLURM_SUBMIT_DIR/$SLURM_JOBID
export WorkDir=$SLURM_SUBMIT_DIR/$SLURM_JOBID
export MOLCAS_WORKDIR="/lustre/isaac/scratch/gjones39"
export MOLCAS="/lustre/isaac/proj/UTK0022/GMJ/OpenMolcas/build"
echo $WorkDir



module purge
module load intel-compilers/2021.2.0
module load hdf5/1.10.8-intel
module load cmake/3.23.2-intel

# DEFINE WorkDir
echo "Starting at $(date)"
echo "Running on hosts: $SLURM_NODELIST"
echo "Running on $SLURM_NNODES nodes."
echo "Running $SLURM_NTASKS tasks."
echo "Current working directory is $(pwd)"

# THE COMMAND

conda activate /lustre/isaac/proj/UTK0022/GMJ/modules/anaconda3/envs/ddcaspt2
pymolcas --new --clean {r}.input -oe {r}.output
# CLEAN-UP AND EXIT
rm -r $SLURM_SUBMIT_DIR/$SLURM_JOBID
echo "Program finished with exit code $? at: $(date)"
""")    

In [6]:

topdir = os.getcwd()



In [9]:
t0 = perf_counter()
geom = Geometry('FeO.xyz')
oxo, iron = geom.atoms[0],geom.atoms[1]

for idxr, r in tqdm(enumerate(radius_range)):

# Create radius subdirectory
    radstr = f"{r:.2f}"
    rad_dir = os.path.join(os.getcwd(),radstr)
    if os.path.exists(rad_dir)==False:
        os.mkdir(rad_dir)
    
        
    # Write xyz
    geom.change_distance(iron,oxo,r)
    geom.write(os.path.join(rad_dir,radstr))
    if idxr==0:
        d = DDCASPT2(rad_dir,'ANO-RCC-VDZP',radstr,10,14,None,previous=None,symmetry=1,spin=4,UHF=True,charge=2,clean=False)(run=False)
    else:            
        
        previous=os.path.join(topdir,f"{radius_range[idxr-1]:.2f}",f"{radius_range[idxr-1]:.2f}.RasOrb")
        print(previous)
        d = DDCASPT2(rad_dir,'ANO-RCC-VDZP',radstr,10,14,None,previous=previous,symmetry=1,spin=4,UHF=True,charge=2,clean=False)(run=False)
    
    gen_run(radstr,rad_dir)
print(perf_counter()-t0)

500it [00:00, 5360.31it/s]

Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp
/home/grierjones/DDCASPT2/iron_oxo_DDCASPT2/cluster/1.40/1.40.RasOrb
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp
/home/grierjones/DDCASPT2/iron_oxo_DDCASPT2/cluster/1.41/1.41.RasOrb
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp
/home/grierjones/DDCASPT2/iron_oxo_DDCASPT2/cluster/1.43/1.43.RasOrb
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp
/home/grierjones/DDCASPT2/iron_oxo_DDCASPT2/cluster/1.44/1.44.RasOrb
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp
/home/grierjones/DDCASPT2/iron_oxo_DDCASPT2/cluster/1.45/1.45.RasOrb
Running on None cores
Found a valid MOLCAS installation at /home/grier